In [2]:
# Import librarries to read images 
import numpy as np
import skimage.io as io
import os
import matplotlib.pyplot as plt
import cv2
import pickle as pkl
from sklearn.metrics import confusion_matrix, jaccard_score
from traditional_utils import *

In [33]:
# Read images from folders A and B
imagesA = []
imagesB = []
labels = []

for i in range(4868):
    if os.path.exists('./dataset/train/A/'+str(i).zfill(4)+'.png'):
        imgA = cv2.imread('./dataset/train/A/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        imgG = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
        
        imagesA.append(imgG)

print(len(imagesA))


3895


In [34]:
for i in range(4868):
    if os.path.exists('./dataset/train/B/'+str(i).zfill(4)+'.png'):
        imgB = cv2.imread('./dataset/train/B/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        imgG = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)
        
        imagesB.append(imgG)

print(len(imagesB))

3895


In [35]:
for i in range(4868):
    if os.path.exists('./dataset/train/OUT/'+str(i).zfill(4)+'.png'):
        label = cv2.imread('./dataset/train/OUT/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        imgG = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        
        labels.append(imgG)

print(len(labels))

3895


In [62]:
# image differencing 

def get_diff_image(imgA, imgB):
    
    diff = imgB - imgA  # difference between the two images
    diff = np.abs(diff) # absolute difference
    threshold = calculate_threshold(diff) # calculate the threshold value for the difference image

    # Apply thresholding
    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    thresholded[thresholded == 255] = 1
    
    
    return thresholded


In [63]:
# Jacard Index
def calculate_jacard_index(imageA, imageB):
    intersection = cv2.bitwise_and(imageA, imageB)
    union = cv2.bitwise_or(imageA, imageB)
    
    if np.sum(union) == 0:
        return 1
    jacard_index = (np.sum(intersection)+ 1e-6) / (np.sum(union) + 1e-6)
    return jacard_index

def compute_iou(image, label):
    flattened_image = image.flatten()
    flattened_label = label.flatten()
    
    tp = np.sum(np.logical_and(flattened_image == 1, flattened_label == 1))
    fp = np.sum(np.logical_and(flattened_image == 1, flattened_label == 0))
    fn = np.sum(np.logical_and(flattened_image == 0, flattened_label == 1))
    
    if (tp + fp + fn) == 0:
        return 1
    else:
        return tp / (tp + fp + fn)


# Dice Coefficient
def calculate_dice_coefficient(imageA, imageB):
    intersection = cv2.bitwise_and(imageA, imageB)
    dice_coefficient = (2 * np.sum(intersection)+ 1e-6) / (np.sum(imageA) + np.sum(imageB) + 1e-6)
    return dice_coefficient


In [64]:
jaccard = 0
for label, imgA, imgB in zip(labels, imagesA, imagesB):
    thresholded = get_diff_image(imgA, imgB)
    label[label == 255] = 1
    # jaccard_index = calculate_jacard_index(label, thresholded)
    jaccard_index = compute_iou(image=thresholded, label=label)
    jaccard += jaccard_index
    dice_coefficient = calculate_dice_coefficient(label, thresholded)
    print('Jacard Index: ', jaccard_index)
    print('Dice Coefficient: ', dice_coefficient)
    print('---------------------------------')

Jacard Index:  0.009802371541501976
Dice Coefficient:  0.019414435725784492
---------------------------------
Jacard Index:  0.04025960855896968
Dice Coefficient:  0.0774030026245464
---------------------------------
Jacard Index:  0.021467798302546182
Dice Coefficient:  0.042033235628444125
---------------------------------
Jacard Index:  0.013153310104529617
Dice Coefficient:  0.025965093368930867
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  1.5573898144280655e-10
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  1.2283503253619516e-10
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  1.7082336860764888e-10
---------------------------------
Jacard Index:  0.06141420298689424
Dice Coefficient:  0.11572146452866322
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  3.183699457757498e-10
---------------------------------
Jacard Index:  0.0058871479032696315
Dice Coefficient:  0.011705384565846804

In [66]:
print('Average Jacard Index: ', jaccard/len(labels))

Average Jacard Index:  0.18112638069609666


In [40]:
# Read images from folders A and B
valA = []
valB = []
valLabels = []

for i in range(4868):
    if os.path.exists('./dataset/val/A/'+str(i).zfill(4)+'.png'):
        imgA = cv2.imread('./dataset/val/A/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        imgG = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
        
        valA.append(imgG)

print(len(valA))


973


In [41]:
for i in range(4868):
    if os.path.exists('./dataset/val/B/'+str(i).zfill(4)+'.png'):
        imgB = cv2.imread('./dataset/val/B/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        imgG = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)
        
        valB.append(imgG)

print(len(valB))

973


In [42]:
for i in range(4868):
    if os.path.exists('./dataset/val/OUT/'+str(i).zfill(4)+'.png'):
        label = cv2.imread('./dataset/val/OUT/'+str(i).zfill(4)+'.png', cv2.IMREAD_COLOR)
        imgG = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        
        valLabels.append(imgG)

print(len(valLabels))

973


In [67]:
jaccard = 0
for label, imgA, imgB in zip(valLabels, valA, valB):
    thresholded = get_diff_image(imgA, imgB)
    label[label == 255] = 1
    jacard_index = compute_iou(image=thresholded, label=label)
    jaccard += jacard_index
    dice_coefficient = calculate_dice_coefficient(label, thresholded)
    print('Jacard Index: ', jacard_index)
    print('Dice Coefficient: ', dice_coefficient)
    print('---------------------------------')

Jacard Index:  0.0
Dice Coefficient:  6.46830530359196e-11
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  1.9440124413017082e-10
---------------------------------
Jacard Index:  0.019516855466084344
Dice Coefficient:  0.038286479379078114
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  2.9489826001330043e-10
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  2.000400079615843e-10
---------------------------------
Jacard Index:  0.0029411764705882353
Dice Coefficient:  0.005865102725041823
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  5.316321102968462e-10
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  1.6608536785150547e-10
---------------------------------
Jacard Index:  0.038013918942628226
Dice Coefficient:  0.07324356306984937
---------------------------------
Jacard Index:  0.0
Dice Coefficient:  6.748093663084682e-11
---------------------------------
Jacard Ind

In [68]:
print('Average Jacard Index: ', jaccard/len(valLabels))

Average Jacard Index:  0.1599255166545284
